##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Noções básicas do TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/basics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este guia fornece uma visão geral rápida dos *princípios básicos do TensorFlow*. Cada seção deste documento é uma visão geral de um tópico maior. Você encontrará links para guias completos no final de cada seção.

TensorFlow é uma plataforma de ponta a ponta para aprendizado de máquina. Ela suporta os seguintes recursos:

- Computação numérica baseada em matriz multidimensional (similar a <a href="https://numpy.org/" class="external">NumPy</a>.)
- GPU e processamento distribuido
- Diferenciação automática
- Construção de modelo, treinamento e exportação
- E mais

## Tensores

O TensorFlow opera em matizes multidimensionais ou *tensores* representados como objetos `tf.Tensor`. Aqui está um tensor bidimensional:

In [ ]:
import tensorflow as tf

x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])

print(x)
print(x.shape)
print(x.dtype)

Os atributos mais importantes de um `tf.Tensor` são seus `shape` (formato) e `dtype`:

- `Tensor.shape`: retorna o tamanho do tensor ao longo de cada um de seus eixos.
- `Tensor.dtype`: retorna o tipo de todos os elementos no tensor.

O TensorFlow implementa operações matemáticas padrão nos tensores, bem como muitas operações específicas para aprendizado de máquina.

Por exemplo:

In [ ]:
x + x

In [ ]:
5 * x

In [ ]:
x @ tf.transpose(x)

In [ ]:
tf.concat([x, x, x], axis=0)

In [ ]:
tf.nn.softmax(x, axis=-1)

In [ ]:
tf.reduce_sum(x)

Observação: normalmente, em qualquer lugar em que uma função do TensorFlow espera um `Tensor` como entrada, a função também aceitará qualquer coisa que possa ser convertida em um `Tensor` usando `tf.convert_to_tensor`. Veja abaixo um exemplo.

In [ ]:
tf.convert_to_tensor([1,2,3])

In [ ]:
tf.reduce_sum([1,2,3])

Realizar grandes cálculos via CPU pode ser lento. Quando configurado de forma correta, o TensorFlow pode usar hardwares aceleradores, como GPUs, para executar operações mais rapidamente.

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **ESTÁ** usando a GPU")
else:
  print("TensorFlow **NÃO ESTÁ** usando a GPU")

Consulte o [Guia de Tensores](tensor.ipynb) para mais detalhes.

## Variáveis

Objetos `tf.Tensor` comuns são imutáveis. Para armazenar pesos de modelo (ou outro estado mutável) em TensorFlow, use `tf.Variable`.

In [ ]:
var = tf.Variable([0.0, 0.0, 0.0])

In [ ]:
var.assign([1, 2, 3])

In [ ]:
var.assign_add([1, 1, 1])

Consulte o [Guia de Variáveis](variable.ipynb) para obter detalhes.

## Diferenciação Automática

O <a href="https://en.wikipedia.org/wiki/Gradient_descent" class="external"><em>método do gradiente descendente</em></a> e algoritmos relacionados são a base do aprendizado de máquina moderno.

Para ativar isso, o TensorFlow implementa a diferenciação automática (autodiff), que usa cálculo diferencial e integral para computar gradientes. Normalmente, isto será usado para calcular o gradiente do *erro* ou *perda* de um modelo em relação a seus pesos.

In [ ]:
x = tf.Variable(1.0)

def f(x):
  y = x**2 + 2*x - 5
  return y

In [ ]:
f(x)

Em `x = 1.0`, `y = f(x) = (1**2 + 2*1 - 5) = -2`.

A derivada de `y` é `y' = f'(x) = (2*x + 2) = 4`. O TensorFlow pode calcular isto automaticamente:

In [ ]:
with tf.GradientTape() as tape:
  y = f(x)

g_x = tape.gradient(y, x)  # g(x) = dy/dx

g_x

Este exemplo simplificado apenas pega a derivada em relação a um único escalar (`x`), mas o TensorFlow pode calcular a gradiente em relação a qualquer número de tensores não escalares simultaneamente.

Consulte o [guia Autodiff](autodiff.ipynb) para mais detalhes.

## Grafos e tf.function

Embora você possa utilizar o TensorFlow de forma interativa como qualquer biblioteca Python, o TensorFlow também fornece ferramentas para:

- **Otimização de desempenho**: para acelerar treinamento e inferência.
- **Exportação**: para que você possa salvar o seu modelo quando ele terminar o treinamento.

Isto requer que você use `tf.function` para separar o seu código TensorFlow puro do Python.

In [ ]:
@tf.function
def my_func(x):
  print('Tracing.\n')
  return tf.reduce_sum(x)

A primeira vez que você executa `tf.function`, embora seja executada em Python, ela captura um grafo completo e otimizado que representa as computações do TensorFlow feitas dentro da função.

In [ ]:
x = tf.constant([1, 2, 3])
my_func(x)

Nas chamadas subsequentes, o TensorFlow executa apenas o grafo otimizado, ignorando todos os passos não relacionados ao TensorFlow. Abaixo, observe que `my_func` não mostra *rastreamento*, pois `print` é uma função do Python, e não uma função do TensorFlow.

In [ ]:
x = tf.constant([10, 9, 8])
my_func(x)

Um grafo pode não ser reutilizável para entradas com uma *assinatura* diferente (`shape` e `dtype`), portanto, um novo grafo é gerado:

In [ ]:
x = tf.constant([10.0, 9.1, 8.2], dtype=tf.float32)
my_func(x)

Esses grafos capturados fornecem dois benefícios:

- Em muitos casos, eles fornecem uma aceleração significativa na execução (embora não neste exemplo trivial).
- Você pode exportar esses grafos, usando `tf.saved_model`, para executar em outros sistemas como um [servidor](https://www.tensorflow.org/tfx/serving/docker) ou um [dispositivo móvel](https://www.tensorflow.org/lite/guide), sem necessidade de instalação do Python.

Consulte [Introdução aos grafos](intro_to_graphs.ipynb) para mais detalhes.

## Módulos, camadas e modelos

`tf.Module` é uma classe para gerenciar seus objetos `tf.Variable` e os objetos `tf.function` que operam neles. A classe `tf.Module` é necessária para dar suporte a dois recursos significativos:

1. Você pode salvar e restaurar os valores de suas variáveis usando `tf.train.Checkpoint`. Isso é útil durante o treinamento, pois é rápido salvar e restaurar o estado de um modelo.
2. Você pode importar e exportar os valores `tf.Variable` *e* os grafos `tf.function` usando `tf.saved_model`. Isto permite que você execute seu modelo independentemente do programa Python que o criou.

Aqui está um exemplo completo de exportação de um objeto `tf.Module` simples:

In [ ]:
class MyModule(tf.Module):
  def __init__(self, value):
    self.weight = tf.Variable(value)

  @tf.function
  def multiply(self, x):
    return x * self.weight

In [ ]:
mod = MyModule(3)
mod.multiply(tf.constant([1, 2, 3]))

Salve o `Module`:

In [ ]:
save_path = './saved'
tf.saved_model.save(mod, save_path)

O SavedModel resultante é independente do código que o criou. Você pode carregar um SavedModel do Python, outros mapeamentos de linguagem ou [TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker). Você também pode convertê-lo para ser executado com [TensorFlow Lite](https://www.tensorflow.org/lite/guide) ou [TensorFlow JS](https://www.tensorflow.org/js/guide).

In [ ]:
reloaded = tf.saved_model.load(save_path)
reloaded.multiply(tf.constant([1, 2, 3]))

As classes `tf.keras.layers.Layer` e `tf.keras.Model` são construídas sobre `tf.Module` fornecendo funcionalidade adicional e métodos convenientes para construir, treinar e salvar modelos. Algumas delas são demonstradas na próxima seção.

Consulte [Introdução aos módulos](intro_to_modules.ipynb) para mais detalhes.

## Loops de treinamento

Agora junte tudo isso para construir um modelo básico e treiná-lo do zero.

Primeiro, crie alguns dados de exemplo. Isso gera uma nuvem de pontos que segue vagamente uma curva quadrática:

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

matplotlib.rcParams['figure.figsize'] = [9, 6]

In [ ]:
x = tf.linspace(-2, 2, 201)
x = tf.cast(x, tf.float32)

def f(x):
  y = x**2 + 2*x - 5
  return y

y = f(x) + tf.random.normal(shape=[201])

plt.plot(x.numpy(), y.numpy(), '.', label='Data')
plt.plot(x, f(x), label='Ground truth')
plt.legend();

Crie um modelo quadrático com pesos inicializados aleatoriamente e um bias (tendência):

In [ ]:
class Model(tf.Module):

  def __init__(self):
    # Randomly generate weight and bias terms
    rand_init = tf.random.uniform(shape=[3], minval=0., maxval=5., seed=22)
    # Initialize model parameters
    self.w_q = tf.Variable(rand_init[0])
    self.w_l = tf.Variable(rand_init[1])
    self.b = tf.Variable(rand_init[2])
  
  @tf.function
  def __call__(self, x):
    # Quadratic Model : quadratic_weight * x^2 + linear_weight * x + bias
    return self.w_q * (x**2) + self.w_l * x + self.b

Primeiro, observe o desempenho do seu modelo antes do treino:

In [ ]:
quad_model = Model()

In [ ]:
def plot_preds(x, y, f, model, title):
  plt.figure()
  plt.plot(x, y, '.', label='Data')
  plt.plot(x, f(x), label='Ground truth')
  plt.plot(x, model(x), label='Predictions')
  plt.title(title)
  plt.legend()

In [ ]:
plot_preds(x, y, f, quad_model, 'Before training')

Agora, defina uma perda para o seu modelo:

Dado que este modelo se destina a prever valores contínuos, o erro quadrático médio (MSE) é uma boa escolha para a função de perda. Dado um vetor de previsões, $\hat{y}$, e um vetor de alvos verdadeiros, $y$, o MSE é definido como a média das diferenças quadradas entre os valores previstos e a verdade básica.

$MSE = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}_i -y_i)^2$

In [ ]:
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

Escreva um loop de treinamento básico para o modelo. O loop fará uso da função de perda MSE e seus gradientes em relação à entrada para atualizar iterativamente os parâmetros do modelo. O uso de minilotes para treinamento garante eficiência de memória e convergência mais rápida. A API `tf.data.Dataset` possui funções úteis para lote e embaralhamento.

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=x.shape[0]).batch(batch_size)

In [ ]:
# Set training parameters
epochs = 100
learning_rate = 0.01
losses = []

# Format training loop
for epoch in range(epochs):
  for x_batch, y_batch in dataset:
    with tf.GradientTape() as tape:
      batch_loss = mse_loss(quad_model(x_batch), y_batch)
    # Update parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, quad_model.variables)
    for g,v in zip(grads, quad_model.variables):
        v.assign_sub(learning_rate*g)
  # Keep track of model loss per epoch
  loss = mse_loss(quad_model(x), y)
  losses.append(loss)
  if epoch % 10 == 0:
    print(f'Mean squared error for step {epoch}: {loss.numpy():0.3f}')

# Plot model results
print("\n")
plt.plot(range(epochs), losses)
plt.xlabel("Epoch")
plt.ylabel("Mean Squared Error (MSE)")
plt.title('MSE loss vs training iterations');

Agora observe o desempenho do seu modelo após o treinamento:

In [ ]:
plot_preds(x, y, f, quad_model, 'After training')

Isso está funcionando, mas lembre-se de que implementações de utilitários de treinamento comuns já estão disponíveis no módulo `tf.keras`. Portanto, considere usá-los antes de escrever o seu próprio. Para começar, os métodos `Model.compile` e `Model.fit` implementam um loop de treinamento para você:

Comece criando um modelo sequencial em Keras usando `tf.keras.Sequential`. Uma das camadas Keras mais simples é a camada densa, que pode ser instanciada com `tf.keras.layers.Dense`. A camada densa é capaz de aprender relações lineares multidimensionais da forma $\mathrm{Y} = \mathrm{W}\mathrm{X} + \vec{b}$. Para aprender uma equação não linear da forma $w_1x^2 + w_2x + b$, a entrada da camada densa deve ser uma matriz de dados com $x^2$ e $x$ como características. A camada lambda, `tf.keras.layers.Lambda`, pode ser usada para realizar essa transformação de empilhamento.

In [ ]:
new_model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.stack([x, x**2], axis=1)),
    tf.keras.layers.Dense(units=1, kernel_initializer=tf.random.normal)])

In [ ]:
new_model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01))

history = new_model.fit(x, y,
                        epochs=100,
                        batch_size=32,
                        verbose=0)

new_model.save('./my_new_model')

Observe o desempenho do seu modelo Keras após o treinamento:

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('Epoch')
plt.ylim([0, max(plt.ylim())])
plt.ylabel('Loss [Mean Squared Error]')
plt.title('Keras training progress');

In [ ]:
plot_preds(x, y, f, new_model, 'After Training: Keras')

Consulte [Loops de treinamento básico](basic_training_loops.ipynb) e o [guia Keras](https://www.tensorflow.org/guide/keras) para obter mais detalhes.